In [ ]:
import requests
import json
import os
import pydicom
from pydicom.dataelem import DataElement
import base64
import glob
from pathlib import Path
import matplotlib.pyplot as plt
from presidio_image_redactor import DicomImageRedactorEngine
import datetime
from PIL import Image, ImageDraw, ImageFont
import numpy as np

In [ ]:
import cv2
import numpy as np

def draw_text_on_image_opencv(image_array, text, position=(10, 30), font_scale=1, font_color=(255, 255, 255), thickness=2, font=cv2.FONT_HERSHEY_SIMPLEX):
    """
    Draws text on a NumPy array representing an image using OpenCV.

    Args:
        image_array (numpy.ndarray): The image as a NumPy array (e.g., shape (height, width, 3) for RGB).
        text (str): The text to draw.
        position (tuple): The (x, y) coordinates of the text's bottom-left corner.
        font_scale (float): The font scale factor.
        font_color (tuple): The BGR color of the text (e.g., (0, 0, 255) for red).
        thickness (int): The thickness of the text lines.
        font (int): The OpenCV font type (e.g., cv2.FONT_HERSHEY_SIMPLEX).

    Returns:
        numpy.ndarray: The modified image as a NumPy array.
    """
    try:
        # OpenCV uses BGR color format, so we reverse the RGB tuple
        bgr_color = (font_color[2], font_color[1], font_color[0])

        # Draw the text using cv2.putText()
        cv2.putText(image_array, text, position, font, font_scale, bgr_color, thickness)

        return image_array

    except Exception as e:
        print(f"Error drawing text: {e}")
        return image_array


def draw_text_on_image_opencv_adaptive(image_array, text, position_percent=(0.01, 0.05), font_scale_percent=0.01, font_color=(0, 0, 0), thickness_percent=0.002, font=cv2.FONT_HERSHEY_SIMPLEX):
    """
    Draws text on a NumPy array representing an image using OpenCV, adapting to different resolutions.

    Args:
        image_array (numpy.ndarray): The image as a NumPy array (e.g., shape (height, width, 3) for RGB).
        text (str): The text to draw.
        position_percent (tuple): The (x, y) coordinates of the text's bottom-left corner, as percentages of image dimensions.
        font_scale_percent (float): The font scale factor, as a percentage of the image height.
        font_color (tuple): The BGR color of the text (e.g., (0, 0, 255) for red).
        thickness_percent (float): The thickness of the text lines, as a percentage of the image height.
        font (int): The OpenCV font type (e.g., cv2.FONT_HERSHEY_SIMPLEX).

    Returns:
        numpy.ndarray: The modified image as a NumPy array.
    """
    try:
        height, width = image_array.shape[:2]  # Get image dimensions

        # Calculate absolute pixel positions and font scale/thickness based on percentages
        position = (int(width * position_percent[0]), int(height * position_percent[1]))
        font_scale = height * font_scale_percent
        thickness = int(height * thickness_percent)

        bgr_color = (font_color[2], font_color[1], font_color[0])
        cv2.putText(image_array, text, position, font, font_scale, bgr_color, thickness)
        return image_array

    except Exception as e:
        print(f"Error drawing text: {e}")
        return image_array


def draw_text_on_image_opencv(image_array, text, position=(10, 30), font_scale=1, font_color=(255, 255, 255), thickness=2, font=cv2.FONT_HERSHEY_SIMPLEX):
    try:
        bgr_color = (font_color[2], font_color[1], font_color[0])
        cv2.putText(image_array, text, position, font, font_scale, bgr_color, thickness)
        return image_array
    except Exception as e:
        print(f"Error drawing text: {e}")
        return image_array

In [ ]:
# Load in and process your DICOM file as needed
dicom_instance = pydicom.dcmread('dicom/manifest-1617826161202/Pseudo-PHI-DICOM-Data/Pseudo-PHI-001/06-26-2003-NA-XR CHEST AP PORTABLE-96544/1002.000000-NA-96023/1-1.dcm')
_dicom_instance = pydicom.dcmread('dicom/manifest-1617826161202/Pseudo-PHI-DICOM-Data/Pseudo-PHI-021/03-30-1987-NA-MRI ABDOMEN PELVIS WWO CONT-29432/16.000000-LIVER-PELVISHASTESAGPOS-56660/1-09.dcm')

In [ ]:
dicom_instance.pixel_array.shape, _dicom_instance.pixel_array.shape

In [ ]:
dicom_instance.PatientBirthDate="19690414"
dicom_instance.PatientName="Juan Huertas"
_dicom_instance.PatientBirthDate="19690414"
_dicom_instance.PatientName="Juan Huertas"

# Write Text on the Image

In [ ]:
# Parameters for the first image
first_image = draw_text_on_image_opencv(dicom_instance.pixel_array, str(dicom_instance.PatientName), position=(20, 80), font_scale=2.0, font_color=(255, 255, 255), thickness=4)
first_image = draw_text_on_image_opencv(first_image, str(dicom_instance.PatientBirthDate), position=(20, 140), font_scale=2.0, font_color=(255, 255, 255), thickness=4)

In [ ]:
dicom_instance.PixelData  = first_image.tobytes()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(dicom_instance.pixel_array)

In [ ]:
# Parameters for the second image
second_image = draw_text_on_image_opencv(_dicom_instance.pixel_array, str(_dicom_instance.PatientName), position=(10, 30), font_scale=0.5, font_color=(255, 255, 255), thickness=1)
second_image = draw_text_on_image_opencv(second_image, str(_dicom_instance.PatientBirthDate), position=(10, 50), font_scale=0.5, font_color=(255, 255, 255), thickness=1)

In [ ]:
_dicom_instance.PixelData  = second_image.tobytes()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(_dicom_instance.pixel_array)

In [ ]:
position = (20, 80)
font_scale = 2.0
font_color = (0, 0, 0)  # Red (BGR)
thickness = 4
font = cv2.FONT_HERSHEY_SIMPLEX # Change font if desired

In [ ]:
#modified_image_array = draw_text_on_image(dicom_instance.pixel_array, str(dicom_instance.PatientName), position, font_size, text_color, font_path)

modified_image_array = draw_text_on_image_opencv(dicom_instance.pixel_array, str(dicom_instance.PatientName), position, font_scale, font_color, thickness, font)

In [ ]:
dicom_instance.PixelData  = modified_image_array.tostring()

In [ ]:
position = (20, 140)
modified_image_array = draw_text_on_image_opencv(dicom_instance.pixel_array, str(dicom_instance.PatientBirthDate), position, font_scale, font_color, thickness, font)
dicom_instance.PixelData  = modified_image_array.tostring()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(dicom_instance.pixel_array)

In [ ]:
dicom_instance.save_as("modified_dcm.dcm")

In [ ]:
dicom_instance2 = pydicom.dcmread("modified_dcm.dcm")
dicom_instance3 = pydicom.dcmread("1-09.dcm")

In [ ]:
dicom_instance3

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(dicom_instance3.pixel_array)

In [ ]:
dicom_instance3 = pydicom.dcmread("dicom2/0002.DCM")

In [ ]:
dicom_instance3

In [ ]:
list_frames = []
for frame in dicom_instance3.pixel_array:
    modified_image_array = draw_text_on_image_opencv(frame, str(dicom_instance.PatientName), position, 1.0, font_color, thickness, font)
    position = (20, 140)
    modified_image_array = draw_text_on_image_opencv(modified_image_array, str(dicom_instance.PatientBirthDate), position, 1.0, font_color, thickness, font)
    list_frames.append(modified_image_array.tostring())

In [ ]:
list_frames = np.array(list_frames)

In [ ]:
dicom_instance3.PixelData  = list_frames.tostring()

In [ ]:
import numpy as np
import cv2

def visualize_array_as_video(array):
    # Create a VideoWriter object
    height, width = array.shape[1], array.shape[2]
    out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, (width, height), isColor=False)

    for i in range(array.shape[0]):
        frame = (array[i] * 255).astype(np.uint8)  # Convert to uint8
        out.write(frame)

    out.release()
    print("Video saved as output.avi")




In [ ]:
#visualize_array_as_video(list_frames)

In [ ]:
#dicom_instance3

In [ ]:
# plt.figure(figsize=(10,10))
# plt.imshow(dicom_instance3.pixel_array[0])